## Training potential model types on test data

## Based on non-exhaustive research, it appears a few of the most used methods are XGBoost (Extreme Gradient Boosting), Generalized Linear Model, Random Forest Regressor.

### Links
#### KTH Royal Inst. of Tech (Sweden) - Comp. of XGBoost and Random Forest on housing data
https://www.diva-portal.org/smash/get/diva2:1597534/FULLTEXT01.pdf

#### NYU Dis. comparing GLM and RF on housing data, RF ended up with lower error.
https://cs.nyu.edu/media/publications/lowrance_roy.pdf

#### Comparing algorithms for housing predictions
https://www.mdpi.com/2073-445X/11/11/2100/pdf

In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split
from sklearn.ensemble import GradientBoostingRegressor
from sklearn.preprocessing import StandardScaler
from sklearn.preprocessing import OneHotEncoder
from sklearn.metrics import mean_squared_error

In [2]:
# Import Data
sf_df = pd.read_csv("../First Segment Rubric - Sample Data/sf_clean.csv")
sf_df.head()

,price,sqft,beds,bath,laundry,pets,housing_type,parking,hood_district
0,6800,1600.0,2.0,2.0,(a) in-unit,(d) no pets,(c) multi,(b) protected,7.0
1,3500,550.0,1.0,1.0,(a) in-unit,(a) both,(c) multi,(b) protected,7.0
2,5100,1300.0,2.0,1.0,(a) in-unit,(a) both,(c) multi,(d) no parking,7.0
3,9000,3500.0,3.0,2.5,(a) in-unit,(d) no pets,(c) multi,(b) protected,7.0
4,3100,561.0,1.0,1.0,(c) no laundry,(a) both,(c) multi,(d) no parking,7.0


In [5]:
# Inspect data types
sf_df.dtypes

price              int64
sqft             float64
beds             float64
bath             float64
laundry           object
pets              object
housing_type      object
parking           object
hood_district    float64
dtype: object

In [6]:
# List categorical variables
sf_cat = sf_df.dtypes[sf_df.dtypes == "object"].index.tolist()

In [7]:
# Encode cat. variables
enc = OneHotEncoder(sparse=False)

sf_encode = pd.DataFrame(enc.fit_transform(sf_df[sf_cat]))

sf_encode.columns = enc.get_feature_names_out(sf_cat)
sf_encode.head()

,laundry_(a) in-unit,laundry_(b) on-site,laundry_(c) no laundry,pets_(a) both,pets_(b) dogs,pets_(c) cats,pets_(d) no pets,housing_type_(a) single,housing_type_(b) double,housing_type_(c) multi,parking_(a) valet,parking_(b) protected,parking_(c) off-street,parking_(d) no parking
0,1.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,1.0,0.0,1.0,0.0,0.0
1,1.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,1.0,0.0,0.0
2,1.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,1.0
3,1.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,1.0,0.0,1.0,0.0,0.0
4,0.0,0.0,1.0,1.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,1.0


In [8]:
# Merge sf_df and sf_encode
sf_df = sf_df.merge(sf_encode, left_index=True, right_index=True)
sf_df = sf_df.drop(columns=sf_cat, axis=1)
sf_df.head()

,price,sqft,beds,bath,hood_district,laundry_(a) in-unit,laundry_(b) on-site,laundry_(c) no laundry,pets_(a) both,pets_(b) dogs,pets_(c) cats,pets_(d) no pets,housing_type_(a) single,housing_type_(b) double,housing_type_(c) multi,parking_(a) valet,parking_(b) protected,parking_(c) off-street,parking_(d) no parking
0,6800,1600.0,2.0,2.0,7.0,1.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,1.0,0.0,1.0,0.0,0.0
1,3500,550.0,1.0,1.0,7.0,1.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,1.0,0.0,0.0
2,5100,1300.0,2.0,1.0,7.0,1.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,1.0
3,9000,3500.0,3.0,2.5,7.0,1.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,1.0,0.0,1.0,0.0,0.0
4,3100,561.0,1.0,1.0,7.0,0.0,0.0,1.0,1.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,1.0


In [9]:
# Remove target
y = sf_df['price'].values
X = sf_df.drop(columns=['price'], axis=1).values

# Split the preprocessed data into a training and testing dataset
X_train, X_test, y_train, y_test = train_test_split(X,y,random_state=42)

In [14]:
# Scale the training data
scaler = StandardScaler()

# Fit the StandardScaler
X_scaler = scaler.fit(X_train)

# Scale the data
X_train_scaled = X_scaler.transform(X_train)
X_test_scaled = X_scaler.transform(X_test)

## Lightly tweaked Gradient Boosting Regressor (scikit-learn's implementation of Extreme Gradient Boosting)

In [35]:
# Create Gradient Boosting Regressor Tree
gbr = GradientBoostingRegressor(max_depth=2, n_estimators=500, random_state=42)

In [36]:
# Fit to training data
gbr.fit(X_train_scaled, y_train)

GradientBoostingRegressor(max_depth=2, n_estimators=500, random_state=42)

In [37]:
# Calc. error vals in predictions
errors = [mean_squared_error(y_test, y_pred)
         for y_pred in gbr.staged_predict(X_test_scaled)]

In [38]:
# Determine best number of estimators
best_n_estimators = np.argmin(errors) + 1

In [39]:
# Train GBRT based on new n_estimators

gbr = GradientBoostingRegressor(max_depth=2, n_estimators=best_n_estimators, random_state=42)


gbr.fit(X_train_scaled, y_train)

GradientBoostingRegressor(max_depth=2, n_estimators=149, random_state=42)

In [40]:
# Display Score
print(f'Training Score: {gbr.score(X_train_scaled, y_train)}')
print(f'Testing Score: {gbr.score(X_test_scaled, y_test)}')

Training Score: 0.8910574542671243
Testing Score: 0.788986756664602


## Basic Generalized Linear Model with Tweedie distribution

In [114]:
from sklearn.linear_model import TweedieRegressor
tr = TweedieRegressor(power=1.5, alpha=.1)

In [115]:
tr.fit(X_train_scaled, y_train)

TweedieRegressor(alpha=0.1, power=1.5)

In [116]:
tr.score(X_test_scaled, y_test)

0.753987465562659

## Random Forest Regressor

In [50]:
from sklearn.ensemble import RandomForestRegressor
rfr = RandomForestRegressor(max_depth=2, n_estimators=149, random_state=42)

In [51]:
rfr.fit(X_train_scaled,y_train)

RandomForestRegressor(max_depth=2, n_estimators=149, random_state=42)

In [52]:
# Display Score
print(f'Training Score: {rfr.score(X_train_scaled, y_train)}')
print(f'Testing Score: {rfr.score(X_test_scaled, y_test)}')

Training Score: 0.7122362989014643
Testing Score: 0.6914438226307921
